## TRAINING
This notebook has as an objective to use a folder containing the formated .npy files to train a deep learning model that can be used in lieu of an FEA model to any accuracy above random, for a proof of concept that it is possible to do so, such that further research can be done afterward to optimize architecture, hyperparameters and data being fed in.

In [68]:
##  imports

import numpy as np
import torch
import torch.nn as nn
import pathlib
import PREPROCESSING_splitting as split

In [131]:
## first, create a class to load the files that are to be fed to the neural network, 
## for both the inputs and the outputs. To avoid confusion I'll refer to the inputs to the 
## FEA model as 'bound_conds' (boundary conditions), and the outputs as 'targets', while what 
## is fed into the neural network will be called an "input", and the output of the neural network "prediction"
## this function should probably be transformed into a dataloader later for a larger dataset, 
## but for now we'll keep it like this

def get_dataset(dataset_path, glob_parameter = '*.npy'):
    # concatenates all samples into a list of boundary conditions and a list of targets
    
    # set paths
    bound_cond_path = pathlib.Path(dataset_path, 'input')
    targets_path = pathlib.Path(dataset_path, 'output')

    test = pathlib.Path('D:/')
    
    
    
    
    # check if folder path is correct
    if bound_cond_path.is_dir() and targets_path.is_dir():
        print('path contains \'input\' and \'output\'')
        pass
    else:
        raise Exception (f'Argument dataset_path: {dataset_path} should contain folders ..\input and ..\output. Please check path')
    
    #create iterators for files
    bound_cond_iterator = bound_cond_path.glob(glob_parameter)
    targets_iterator = targets_path.glob(glob_parameter)
    
    #zip them to ensure that they are going through the same samples 
    samples_iterator = zip(bound_cond_iterator, targets_iterator)
    
    boundary_conditions = np.array([])
    targets = np.array([])
    
    for boundary_condition_files, targets_files in samples_iterator:
        if split.get_number(boundary_condition_files.name) == split.get_number(targets_files.name):
            
            boundary_conditions_temp = np.load(boundary_condition_files)
            targets_temp = np.load(targets_files)
            
            #start array if it hasn't been started yet
            if boundary_conditions.size == 0 and targets.size == 0:
                boundary_conditions = boundary_conditions_temp
                targets = targets_temp
            else:
                boundary_conditions = np.concatenate((boundary_conditions, boundary_conditions_temp), axis = 0)
                targets = np.concatenate((targets, targets_temp), axis = 0)
        else:
            raise Exception('the samples in the iterator are not synced')
    
    return torch.from_numpy(boundary_conditions).float(), torch.from_numpy(targets).float()
    

In [132]:
folders_path =  pathlib.Path('D:/Ansys Simulations/Project/2D/data/proof_of_concept/scaled/arrays')
dataset = get_dataset(dataset_path = folders_path)
print(dataset[0].shape)
print(dataset[1].shape)

path contains 'input' and 'output'
torch.Size([102, 7, 32, 32])
torch.Size([102, 4, 32, 32])


In [ ]:
class TwoD_Dataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, npy_file, root_dir, transform=None):
        """
        Args:
            npy_file (string): Path to the npy files.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.boundary_conditions = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.landmarks_frame.iloc[idx, 0])
        image = io.imread(img_name)
        landmarks = self.landmarks_frame.iloc[idx, 1:]
        landmarks = np.array([landmarks])
        landmarks = landmarks.astype('float').reshape(-1, 2)
        sample = {'image': image, 'landmarks': landmarks}

        if self.transform:
            sample = self.transform(sample)

        return sample


In [151]:
## define the neural network's general shape

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        
        ## convolutional layers
        self.conv1 = nn.Conv2d(in_channels = 7, out_channels = 14, kernel_size = 3, padding = 1)
        self.conv2 = nn.Conv2d(in_channels = 14, out_channels = 16, kernel_size = 3, padding = 1)
        self.deconv1 = nn.ConvTranspose2d(in_channels = 16, out_channels = 14, kernel_size = 3, padding = 1)
        self.deconv2 = nn.ConvTranspose2d(in_channels = 14, out_channels = 10, kernel_size = 3, padding = 1)
        self.deconv3 = nn.ConvTranspose2d(in_channels = 10, out_channels = 7, kernel_size = 3, padding = 1)
        self.deconv4 = nn.ConvTranspose2d(in_channels = 7, out_channels = 3, kernel_size = 3, padding = 1)
        
        ## activation
        self.hardtanh = nn.Hardtanh()
        
        
        ##possible for later: MultiheadAttention
        
    def forward(self, boundary_conditions):
        #print(boundary_conditions.shape)
        x = self.conv1(boundary_conditions)
        x = self.hardtanh(x)
        #print(x.shape)
        x = self.conv2(x)
        x = self.hardtanh(x)
        #print(x.shape)
        x = self.deconv1(x)
        x = self.hardtanh(x)
       #print(x.shape)
        x = self.deconv2(x)
        x = self.hardtanh(x)
        #print(x.shape)
        x = self.deconv3(x)
        x = self.hardtanh(x)
       # print(x.shape)
        x = self.deconv4(x)
        x = self.hardtanh(x)
        #print(x.shape)
        return x
    
net = ConvNet().float()

## test to see if getting the correct size
net.forward(dataset[0][0:1,:,:,:])

tensor([[[[ 0.0899,  0.1401,  0.1367,  ...,  0.1344,  0.1332,  0.1642],
          [ 0.0275,  0.0331,  0.0501,  ...,  0.0514,  0.0402,  0.0876],
          [ 0.0151,  0.0327,  0.0461,  ...,  0.0434,  0.0454,  0.0939],
          ...,
          [ 0.0117,  0.0397,  0.0714,  ...,  0.0555,  0.0526,  0.0994],
          [ 0.0217,  0.0424,  0.0708,  ...,  0.0494,  0.0496,  0.0974],
          [ 0.0364,  0.0131,  0.0366,  ...,  0.0214,  0.0174,  0.0634]],

         [[ 0.1623,  0.1757,  0.1672,  ...,  0.1696,  0.1659,  0.1744],
          [ 0.1989,  0.1802,  0.1755,  ...,  0.1765,  0.1681,  0.1373],
          [ 0.1989,  0.1784,  0.1785,  ...,  0.1796,  0.1698,  0.1390],
          ...,
          [ 0.1771,  0.1524,  0.1432,  ...,  0.1626,  0.1582,  0.1340],
          [ 0.1951,  0.1699,  0.1705,  ...,  0.1940,  0.1766,  0.1503],
          [ 0.2255,  0.2246,  0.2194,  ...,  0.2200,  0.2151,  0.1395]],

         [[-0.2076, -0.2259, -0.2587,  ..., -0.2501, -0.2422, -0.2105],
          [-0.1878, -0.2640, -

In [152]:
## define criterion
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(params = net.parameters(), lr=0.0001, betas=(0.9, 0.999))

In [154]:
## define a general training loop
losses = np.array([])
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    boundary_conditions, targets = dataset[0][:90,:,:,:], dataset[1][:90,:,:,:]
    #print(type(boundary_conditions))
    for i, bc in enumerate(boundary_conditions):
        tgt = targets[i, 1:4, :, :].unsqueeze(0)
        bc = bc.unsqueeze(0)
        #print(i, bc.shape, tgt.shape)

        # get the inputs; data is a list of [boundary_conditions, targets]
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        prediction = net(bc)
        loss = criterion(prediction, tgt)
        #print(loss)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 0:    # print every 10 mini-batches
            running_val_loss = 0.0
            with torch.no_grad():
                boundary_conditions_val, targets_val = dataset[0][90:,:,:,:], dataset[1][90:,:,:,:]
                for ii, bc_val in enumerate(boundary_conditions_val):
                    tgt_val = targets_val[i, 1:4, :, :].unsqueeze(0)
                    bc_val = bc_val.unsqueeze(0)
                    prediction_val = net(bc_val)
                    val_loss = criterion(prediction_val, tgt_val)
                    running_val_loss+=val_loss.item()
                
            print(f'{epoch + 1} {i + 1} {running_loss} {running_val_loss/10}')
            np.append(losses, [running_loss, running_val_loss])
            running_loss = 0.0
            

print('Finished Training')

1 1 0.03659322112798691 0.043989747017622
2 1 0.0020094902720302343 0.0022937262430787085
3 1 0.0015545941423624754 0.0018099371693097055
4 1 0.001284114783629775 0.0015103648998774587
5 1 0.001086575910449028 0.0012802096316590905
6 1 0.0009334570495411754 0.0010969194176141174
7 1 0.0008097579120658338 0.0009486907161772251
8 1 0.0007059931522235274 0.0008243066491559148
9 1 0.000616582459770143 0.000717484694905579
10 1 0.0005400271038524806 0.000627140054712072
11 1 0.00047517954953946173 0.0005520317761693149
12 1 0.0004204793367534876 0.0004899854509858414
13 1 0.0003749725001398474 0.0004391036054585129
14 1 0.0003376926470082253 0.00039748791023157536
15 1 0.00030743962270207703 0.00036348945286590606
16 1 0.00028301571728661656 0.0003358305606525391
17 1 0.00026335837901569903 0.0003134726357529871
18 1 0.0002475546789355576 0.00029551324550993743
19 1 0.00023482968390453607 0.0002811559199471958
20 1 0.0002245364448754117 0.00026970973558491094
Finished Training


In [155]:
## attempt to plot losses over time